In [1]:
import numpy as np 
import pandas as pd

!pip install flair
from flair.models import TextClassifier
from flair.data import Sentence

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 KB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 82.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 67.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 KB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2

In [2]:
winter1_2 = pd.read_csv("winter1_2.csv")
winter1_2 = winter1_2[["Win 1_2", "Category 1", "Category 2"]]
winter1_2

,Win 1_2,Category 1,Category 2
0,Took the time to do some meal prep last night ...,Physical,NaN
1,1. Select one of the activities that resonates...,Physical,Mental
2,"Today, I cooked myself lunch and it was delici...",Physical,NaN
3,I worked on my Paint by Diamond piece.,Mental,NaN
4,I took a 20 minutes walk from the train statio...,Physical,NaN
...,...,...,...
103,I worked out arms and core for 30 minutes,Physical,NaN
104,I chose to participate in social connectedness...,Community,NaN
105,I played fetch with my dogs and gave both of t...,Emotional,NaN
106,I took a walk to get coffee and enjoyed the co...,Physical,Mental


In [3]:
all_dups = pd.DataFrame()
for i in range(len(winter1_2)):
    if not pd.isnull(winter1_2["Category 2"][i]):
        temp = winter1_2[["Win 1_2", "Category 2"]].iloc[i]
        all_dups = all_dups.append(temp)
    else:
        continue
all_dups = all_dups[["Win 1_2", "Category 2"]]
all_dups = all_dups.rename(columns={"Category 2": "Category"})
all_dups

,Win 1_2,Category
1,1. Select one of the activities that resonates...,Mental
6,"Since we started school completely virtually, ...",Community
10,I connected with my environment by taking a wa...,Environmental
11,"I went to the gym multiple times, specifically...",Environmental
16,I went to the IMA in the middle of the day so ...,Environmental
17,I listened to a podcast that wasn't about the ...,Emotional
25,"I took a walk in my neighborhood, then wrote a...",Emotional
27,I went to LA Fitness and exercised today.,Environmental
30,Happy New Year 2022! My goal in this year is ...,Mental
33,I took a walk with my husband and our dog in o...,Community


In [4]:
winter1_2 = winter1_2.rename(columns={"Category 1": "Category"})
winter1_2 = winter1_2[["Win 1_2", "Category"]]
winter1_2 = winter1_2.append(all_dups)
winter1_2

,Win 1_2,Category
0,Took the time to do some meal prep last night ...,Physical
1,1. Select one of the activities that resonates...,Physical
2,"Today, I cooked myself lunch and it was delici...",Physical
3,I worked on my Paint by Diamond piece.,Mental
4,I took a 20 minutes walk from the train statio...,Physical
...,...,...
88,I took a walk from my apartment over to uvilla...,Environmental
92,I reached out to a friend planning how we can ...,Mental
101,I try to take a time off from everything relat...,Spiritual
102,I took a nap for an hour while listening to so...,Emotional


In [5]:
classifier = TextClassifier.load('en-sentiment')
def predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    text = Sentence(sentence)
    #stacked_embeddings.embed(text)
    classifier.predict(text)
    value = text.labels[0].to_dict()['value'] 
    if value == 'POSITIVE':
        result = text.to_dict()['all labels'][0]['confidence']
    else:
        result = -(text.to_dict()['all labels'][0]['confidence'])
    return round(result, 3)

2023-05-21 00:39:22,956 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpmbfat_zg
100%|██████████| 253M/253M [00:11<00:00, 23.6MB/s]2023-05-21 00:39:34,678 copying /tmp/tmpmbfat_zg to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt

2023-05-21 00:39:34,804 removing temp file /tmp/tmpmbfat_zg


In [12]:
test = winter1_2["Win 1_2"][3]
test

'I worked on my Paint by Diamond piece.'

In [13]:
predict(test)

0.999

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443b2d8b-ed93-43a6-bbe3-c3fe69ac80e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>